In [1]:
import numpy as np
import tensorflow as tf 
import tensorflow_hub as hub

In [17]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder-lite/2"
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
#module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)

INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_0:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_1:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_10:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_11:0 

INFO:tensorflow:Initialize variable module_2/SNLI/Classifier/tanh_layer_0/weights:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with SNLI/Classifier/tanh_layer_0/weights
INFO:tensorflow:Initialize variable module_2/global_step:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with global_step


In [22]:
messages = ['Indicies dipping, big names show an  bottom, see if they break or hold these lows #amzn #aapl #ba', 'Increased our stake in #aapl by 5%. Love the foldable idea, but don’t think people will pay $2,000 to be the first guinea pigs. #samsung #foldable']

In [24]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
            (str(x) for x in message_embedding))
        print("Embedding: [{}]\n".format(message_embedding_snippet))

Message: Indicies dipping, big names show an  bottom, see if they break or hold these lows #amzn #aapl #ba
Embedding size: 512
Embedding: [0.035551752895116806, -0.02141728438436985, -0.0320415161550045, 0.004606718197464943, -0.0781623125076294, -0.017017973586916924, 0.08020199090242386, -0.015466340817511082, -0.03443828970193863, 0.043730854988098145, 0.01967061683535576, 0.032048702239990234, -0.010455921292304993, -0.00025252890191040933, 0.035299625247716904, 0.04918111115694046, 0.01607479155063629, -0.05315680056810379, -0.035091936588287354, 0.016033316031098366, 0.012439241632819176, -0.037711285054683685, -0.044104620814323425, 0.022415967658162117, -0.08418199419975281, -0.042590945959091187, -0.0468602292239666, -0.016102764755487442, 0.010148327797651291, 0.004097165074199438, 0.057600051164627075, -0.012202269397675991, -0.013530867174267769, -0.040123384445905685, 0.07343719899654388, -0.0213327556848526, 0.06467987596988678, 0.001451648655347526, -0.001113493810407817

In [25]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), 
    	signature="default", as_dict=True)["default"]

In [26]:
class Model:
    
    def __init__(self):
        self.model = self.init_model()
        
    def init_model(self):
        input_text = layers.Input(shape=(1,), dtype=tf.string)
        embedding = layers.Lambda(self._UniversalEmbedding,
            output_shape=(embed_size,))(input_text)
        dense = layers.Dense(256, activation='relu')(embedding)
        pred = layers.Dense(category_counts, activation='softmax')(dense)
        model = Model(inputs=[input_text], outputs=pred)
        model.compile(loss='categorical_crossentropy', 
            optimizer='adam', metrics=['accuracy'])
        return model
    
    def _UniversalEmbedding(self, x):
        return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]